In [2]:
import os
from dotenv import load_dotenv
# ✅ Step 1: Load environment variables
load_dotenv()
# Make sure to set your OpenAI API key in the .env file or directly here

# ✅ Step 2: Import libraries
from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain import hub
import openai
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


openai.api_key = os.getenv("OPENAI_API_KEY")  # Ensure you have set this in your .env file
embedding = OpenAIEmbeddings(openai_api_key=openai.api_key)
vectorstore = FAISS.load_local("nba_vector_db", embeddings=embedding, allow_dangerous_deserialization=True)

C:\Users\16343\AppData\Local\Temp\ipykernel_37488\3650198341.py:18: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(openai_api_key=openai.api_key)


In [3]:
## Define the Retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})\
## Define the prompt
prompt = hub.pull("jclemens24/rag-prompt")

c:\Users\16343\anaconda3\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [4]:
def format_docs(docs):
 return "\n\n".join(doc.page_content for doc in docs)

In [5]:
## Define the LLM
llm = ChatOpenAI(model_name="gpt-4o-mini")

In [6]:
## Define the chain
rag_chain = (
    {"context": retriever | format_docs,
    "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [7]:
rag_chain.invoke("How many championship does Klay Thompson has ?")

'Klay Thompson has 4 NBA championships.'